In [25]:
# import everything ml
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import string
from wordcloud import WordCloud
import nltk
from collections import Counter
import contractions
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [26]:
true_dataset = pd.read_csv("./data/fakenews/True.csv")
false_dataset = pd.read_csv("./data/fakenews/Fake.csv")

In [27]:
def combine_and_randomize_datasets(true_dataset, false_dataset):
    true_arr = np.ones(len(true_dataset))
    false_arr = np.zeros(len(false_dataset))

    true_dataset['real'] = true_arr
    false_dataset['real'] = false_arr
    combined_dataset = pd.concat([true_dataset, false_dataset], ignore_index=True)
    combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

    return combined_dataset

combined = combine_and_randomize_datasets(true_dataset, false_dataset)

In [28]:


def clean_text(text):
    text = contractions.fix(text)                         # Expand contractions
    text = text.encode("ascii", errors="ignore").decode() # Remove non-ASCII chars
    text = text.lower()                                   # Lowercase
    text = re.sub(r"http\S+|www.\S+", "", text)           # Remove URLs
    text = re.sub(r"<.*?>", "", text)                     # Remove HTML tags
    text = re.sub(r"[^a-z\s]", "", text)                  # Keep only letters and spaces
    text = re.sub(r"\b\w{1}\b", "", text)                 # Remove single-letter words
    text = re.sub(r"\s+", " ", text).strip()              # Remove extra spaces
    words = text.split()
    # words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)



In [29]:
words_dict = {}

# def process_words_into_dict(text, words_dict):
#     for word in text.split():
#         if word not in words_dict:
#             words_dict[word] = 1
#         else:
#             words_dict[word] += 1

# for text in combined['text']:
#     text = clean_text(text)
# tfidf = calculate_tfidf(combined['text'])

X = combined['text']
y = combined['real'].to_list()

test_x = X[:20]
X = X[20:]

test_y=y[:20]
y=y[20:]

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
X_tfidf = vectorizer.fit_transform(X)  # shape (n_samples, n_features)
print(f'shape {X_tfidf.shape}')
# print the first little bit of it
print(X_tfidf[0:5])

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# --- Convert to dense and then to PyTorch tensors
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


shape (44878, 5000)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 436 stored elements and shape (5, 5000)>
  Coords	Values
  (0, 2696)	0.0856331519315061
  (0, 3714)	0.03218339038274704
  (0, 3379)	0.035866123219599366
  (0, 2992)	0.048818949713531234
  (0, 1392)	0.025816555451159292
  (0, 4624)	0.18653841290348563
  (0, 3838)	0.08263576374158307
  (0, 241)	0.5805641280018509
  (0, 1928)	0.03515734454806418
  (0, 4947)	0.07542247043249736
  (0, 2163)	0.062404155623856455
  (0, 2968)	0.02792672738901728
  (0, 4981)	0.04219733242686691
  (0, 2675)	0.1382507087416752
  (0, 3706)	0.047721148756005465
  (0, 729)	0.4300982736464058
  (0, 4067)	0.12027643794151972
  (0, 137)	0.06942980756828003
  (0, 4420)	0.09066964047456684
  (0, 1164)	0.057559095000476124
  (0, 1288)	0.06760391599741494
  (0, 1735)	0.04920815635446646
  (0, 4061)	0.07935319752780719
  (0, 4123)	0.06584017182014029
  (0, 4305)	0.06150473267115355
  :	:
  (4, 3064)	0.0474569714151201
  (4, 4519)	0.04771782945

In [30]:
# --- PyTorch Dataset
class NewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y.unsqueeze(1)  # Per sample

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = NewsDataset(X_train_tensor, y_train_tensor)
test_dataset = NewsDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)


In [31]:
# --- Simple Feedforward Model
# class FakeNewsClassifier(nn.Module):
#     def __init__(self, input_dim):
#         super().__init__()
#         self.fc1 = nn.Linear(input_dim, 100)
#         self.fc2 = nn.Linear(100, 1)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = torch.sigmoid(self.fc2(x))
#         return x

class FakeNewsClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 1000),
            nn.ReLU(),
            nn.Linear(1000,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.classifier(x)
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
model = FakeNewsClassifier(input_dim=5000).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [32]:
# --- Training Loop
for epoch in range(50):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")


Epoch 1 - Loss: 69.5751
Epoch 2 - Loss: 13.1051
Epoch 3 - Loss: 7.0878
Epoch 4 - Loss: 4.8092
Epoch 5 - Loss: 3.8050
Epoch 6 - Loss: 2.9222
Epoch 7 - Loss: 1.8810
Epoch 8 - Loss: 1.4113
Epoch 9 - Loss: 2.4236
Epoch 10 - Loss: 1.2800
Epoch 11 - Loss: 2.2734
Epoch 12 - Loss: 1.1419
Epoch 13 - Loss: 0.9354
Epoch 14 - Loss: 0.3775
Epoch 15 - Loss: 3.4792
Epoch 16 - Loss: 3.4804
Epoch 17 - Loss: 3.4369
Epoch 18 - Loss: 3.4365
Epoch 19 - Loss: 3.4362
Epoch 20 - Loss: 3.4255
Epoch 21 - Loss: 3.4432
Epoch 22 - Loss: 3.4099
Epoch 23 - Loss: 3.4507
Epoch 24 - Loss: 3.3986
Epoch 25 - Loss: 3.4957
Epoch 26 - Loss: 3.4135
Epoch 27 - Loss: 3.4326
Epoch 28 - Loss: 3.4737
Epoch 29 - Loss: 3.5055
Epoch 30 - Loss: 3.4057
Epoch 31 - Loss: 3.5092
Epoch 32 - Loss: 8.5783
Epoch 33 - Loss: 4.6795
Epoch 34 - Loss: 2.6627
Epoch 35 - Loss: 1.4890
Epoch 36 - Loss: 1.4381
Epoch 37 - Loss: 1.5918
Epoch 38 - Loss: 1.6042
Epoch 39 - Loss: 1.6415
Epoch 40 - Loss: 1.5997
Epoch 41 - Loss: 1.6434
Epoch 42 - Loss: 1.6700

In [33]:
# --- Evaluation
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)  # Ensure inputs are on model's device
    y_test_tensor = y_test_tensor.to(device)  # Ensure labels are on the same device

    preds = model(X_test_tensor).squeeze()
    #print(f"average pred: {preds.mean().item()}")
    preds_label = (preds > 0.5).int()
    accuracy = (preds_label == y_test_tensor.int()).float().mean()
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9905


In [38]:
def predict_news_article(article_text, vectorizer, model):
    model.eval()
    with torch.no_grad():
        # Vectorize input string
        tfidf_vector = vectorizer.transform([article_text])
        tfidf_tensor = torch.tensor(tfidf_vector.toarray(), dtype=torch.float32).to(device)

        # Forward pass
        output = model(tfidf_tensor)
    
        return output, output > 0.5

# test_article = "The government has announced a new plan to build solar-powered roads by 2050..."
# test_article=false_dataset['text'][123]
idx = 0
for i in range(19):
    test_article = test_x[idx]
    test_article_pred = test_y[idx]
    print(f"pred {test_article_pred} testing article: {test_article}")

    prediction, is_true = predict_news_article(test_article, vectorizer, model)
    print(f"Prediction: {prediction} \n Is True {is_true} was actually: {test_article_pred>0.5}")
    idx = idx+1

pred 0.0 testing article: Republican nominee Donald Trump recently surprised everyone when he flip-flopped on the harsh immigration policy he d been promising his racist, hateful supporters since the beginning of his campaign. It turns out, not everyone is taking this news very well   especially the Trump lovers who wanted nothing more than to see millions of illegal immigrants heartlessly removed from the country they built their lives in.On Wednesday s segment of The Glenn Beck Program, a furious Trump supporter shared his reaction to Trump s recent softening on immigration   and it wasn t good. In speaking to host Glenn Beck,  Nate from Virginia  actually delivered a threat to Trump. The guest warned: As long as he does the basic things, the foundational things, which is build a wall, he s not going to have people like me coming after him. Beck wasn t alarmed at first, but was visibly stunned as angry Nate continued his rant. Seemingly speaking for all Trump supporters, Nate said: O